In [15]:
from bayes_sensor import *
from config import VALID_CONFIG
from pprint import pprint
DEFAULT_PROBABILITY_THRESHOLD = 0.5

I wish to describe how the bayesian_sensor operates, lets do an investigation

### Code references
* https://github.com/home-assistant/home-assistant/blob/dev/homeassistant/components/binary_sensor/bayesian.py code
* https://home-assistant.io/components/binary_sensor.bayesian/ docs
* https://github.com/jlmcgehee21/smart_hass#binary-bayes-introspection HA sensor author script for working with bayes sensor  
* https://github.com/home-assistant/home-assistant/tree/a1f238816b6130aee2ac88fe9da54ba8f65225f3 Very early home-assistant commit to better understand HA architechture


### Bayes references
* https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/02-Discrete-Bayes.ipynb Recommended reading
* https://en.wikipedia.org/wiki/Bayes%27_theorem wikipedia on Bayes theorem
* https://en.wikipedia.org/wiki/Bayesian_inference wikipedia on Bayesian inference

### The Bayesian sensor

So you've heard about the [bayesian sensor](https://home-assistant.io/components/binary_sensor.bayesian/) in home-assistant, and would like to know what its good for and how to use it. Lets consider a situation where you would like a sensor which indicates when some event which cannot be directly measured is takign place, for example cooking in the kitchen (henceforth event *A*). We will call this sensor **bayesian_sensor.cooking** and we are happy that it is a binary sensor where if someone is cooking the state is ON, otherwise the state is OFF. We are happy that this sensor should be ON if the probability that someone is cooking is greater than 50% (the threshold). 

I estimate that I spend roughly 10% of my day cooking, so if your were to enter the kitchen at a random time the probability that you would find me cooking is $P(A)$ = 0.1. This probability is reffered to as the *prior* probability, as it is my belief prior to any measurement. Lets say that I have built a DIY sensor for detecting cooking smells and through experimentation I have concluded that it registers ON 70% of the time when I am cooking. I call the ON state of this sensor the event *B* and can now state $P(B|A)$ = 0.7 (i.e the probability that this sensor is ON *given* that I am cooking is 50%). However this sensor is not perfect and registers a false-positive ON 10% of the time, $P(B| \neg A)$ = 0.1 where $\neg A$  is the notation for the event *not* $A$. 

It turns out that we now have enough information to create **bayesian_sensor.cooking**. In particular we can calculate $P(A|B)$, which is the probability that I am cooking (event *A*) *given* that the DIY sensor is ON (event *B*). $P(A|B)$ can be calculated using the Bayes forumla, which is be expressed in words as *the posterior ($P(A|B)$) is proportional to the likelihood ($P(B|A)$) times the prior ($P(A)$)*. 

$$\mathtt{posterior} = \frac{\mathtt{likelihood} \times \mathtt{prior}}{\mathtt{normalization}}$$ 

Using the same notation as wikipedia, the [full expression](https://en.wikipedia.org/wiki/Bayes%27_theorem) for the Bayes formula is:

$$P(A|B) = \frac{P(B|A)\,P(A)}{ P(B|A) P(A) + P(B| \neg A) P(\neg A)}\cdot$$

Here $P(\neg A)$ is the probability *against* A where $P(\neg A) = 1 - P(A)$, and the purpose of the denominator is to ensure that the sum of all posterior probabilities is 1. Bayes formula is implemented in the bayesian_sensor component in the function update_probability():

In [55]:
def update_probability(prior, prob_true, prob_false):
    """Update probability using Bayes' rule."""
    numerator = prob_true * prior
    denominator = numerator + prob_false * (1 - prior)
    probability = numerator / denominator
    return probability

For our example, the prior = $P(A)$ = 0.1, prob_true = $P(B|A)$ = 0.7 and prob_false = $P(B| \neg A)$ = 0.1. Lets calculate the posterior probability ($P(A|B)$) given that we have received a single ON reading from the DIY sensor:

In [23]:
prior = 0.1
prob_true = 0.7
prob_false = 0.1
posterior = update_probability(prior, prob_true, prob_false)
print(round(posterior,2))

0.44


We find that our posterior proability is below the 0.5 (50%) threshold, and **bayesian_sensor.cooking** remains OFF. This isn't surprising since our prior was low to begin with, and our false positive rate is not negligible. However we now have an updated prior in the just calculated posterior. Lets run the calculation with the updated prior of 0.44:

In [26]:
posterior = update_probability(0.44, prob_true, prob_false)
print(round(posterior,2))

0.85


And great news, with only two readings our posterior probability is above the 0.5 threshold and **bayesian_sensor.cooking** now displays ON. Lets now consider the same case but where our DIY sensor is much less sensitive, and only indicates cooking 20% of the time but maintains the 10% false-positive (specificity) rate. How many sensor readings are required now?

In [43]:
prior = 0.1
prob_true = 0.2
prob_false = 0.1
posterior = prior # Our initial conditions
while posterior < 0.5:
    posterior = update_probability(posterior, prob_true, prob_false)
    print(posterior)

0.18181818181818182
0.3076923076923077
0.47058823529411764
0.64


We find that even for a sensor with 10% sensitivity, four consecutive positive readings ('samples') are enough to cross the 50% threshold and trigger **bayesian_sensor.cooking** to ON.

Now, how do we update our posterior if we receive a negative reading?

How about combining the readings from several noisy sensors?

## Bayesian sensor
Lets now import the actual sensor used in HA. This sensor updates as new observations come in, sequentially updating the prior.

In [44]:
VALID_CONFIG

{'device_class': 'binary_device',
 'name': 'in_bed',
 'observations': [{'entity_id': 'sensor.bedroom_motion',
   'id': 0,
   'platform': 'state',
   'prob_given_true': 0.5,
   'to_state': 'on'},
  {'entity_id': 'sun.sun',
   'id': 1,
   'platform': 'state',
   'prob_given_true': 0.7,
   'to_state': 'below_horizon'}],
 'prior': 0.25,
 'probability_threshold': 0.95}

Create a bayesian sensor to explore properties

In [45]:
b_sensor = setup_platform(VALID_CONFIG)

updates are handeled through async_threshold_sensor_state_listener()

In [46]:
pprint(vars(b_sensor))

{'_deviation': False,
 '_device_class': 'binary_device',
 '_name': 'in_bed',
 '_observations': [{'entity_id': 'sensor.bedroom_motion',
                    'id': 0,
                    'platform': 'state',
                    'prob_given_true': 0.5,
                    'to_state': 'on'},
                   {'entity_id': 'sun.sun',
                    'id': 1,
                    'platform': 'state',
                    'prob_given_true': 0.7,
                    'to_state': 'below_horizon'}],
 '_probability_threshold': 0.95,
 'current_obs': OrderedDict(),
 'entity_obs': {'sensor.bedroom_motion': [{'entity_id': 'sensor.bedroom_motion',
                                           'id': 0,
                                           'platform': 'state',
                                           'prob_given_true': 0.5,
                                           'to_state': 'on'},
                                          {'entity_id': 'sun.sun',
                                           'id

In [47]:
b_sensor.device_state_attributes

{'observations': [], 'probability': 0.25, 'probability_threshold': 0.95}

In [48]:
b_sensor.state

'off'

In [54]:
b_sensor.entity_obs.keys()

dict_keys(['sensor.bedroom_motion', 'sun.sun'])

In [50]:
b_sensor.current_obs.values()

odict_values([])